In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import gc

In [ ]:
!pip install pyarrow

In [ ]:
Y=pd.read_parquet("/kaggle/input/icecube-neutrinos-in-deep-ice/train_meta.parquet")

In [ ]:
Y=Y[Y['batch_id']>=520]
Y=Y[Y['batch_id']<=525]
gc.collect()

In [ ]:
def y_to_xyz(batch_y):
    angle_code=batch_y
    angle_code=np.transpose(angle_code)
    angle_code=np.transpose(np.array([np.cos(angle_code[0])*np.sin(angle_code[1]),
                                      np.sin(angle_code[0])*np.sin(angle_code[1]),
                                      np.cos(angle_code[1])]))
    
    return angle_code



In [ ]:
def loadY(bid):
    y=Y[Y['batch_id']==bid]
    y = y[['azimuth', 'zenith']].to_numpy()
    y=y_to_xyz(y)
    return y
def loadX(bid):
    resGraph=pd.read_csv("/kaggle/input/graphnetresults1/results_"+str(bid)+".csv")
    resLSTM=np.load("/kaggle/input/lstmresults1/lstm_batch_"+str(bid)+".npz")['arr_0']
    #print(resGraph)
    resGraph= resGraph[['direction_x','direction_y','direction_z','direction_kappa']].to_numpy()
    y1=Y[Y['batch_id']==bid]
    pulses=y1[['last_pulse_index']].to_numpy()-y1[['first_pulse_index']].to_numpy()
    
    e1=np.expand_dims(np.sum(np.log(resLSTM[:,0:575]+1e-6)*resLSTM[:,0:575],axis=1),axis=-1)
    e2=np.expand_dims(np.sum(np.log(resLSTM[:,576:1151]+1e-6)*resLSTM[:,576:1151],axis=1),axis=-1)
    e3=np.expand_dims(np.sum(np.log(resLSTM[:,1152:1727]+1e-6)*resLSTM[:,1152:1727],axis=1),axis=-1)
    
    x=np.concatenate((resLSTM,resGraph,pulses,e1,e2,e3),axis=1)
    gc.collect()
    return x

In [ ]:
#y=loadY(520)
#y

In [ ]:
#x=loadX(520)
#x

In [ ]:
#x.shape

In [ ]:
def MeanAngErr(y_true, y_pred):
    fcos=tf.math.scalar_mul(-0.99999,tf.keras.losses.cosine_similarity(y_true, y_pred))
    return tf.reduce_mean(tf.math.acos(fcos), axis=-1)  # Note the `axis=-1`
def create_model():
    
    inputs = tf.keras.Input(shape=(1733,))
    
    intermediate = tf.keras.layers.Reshape((1733,1), input_shape=(1733,))(inputs)

    inputs1 = tf.keras.layers.Cropping1D(cropping=(2,0))(intermediate)
    inputs1 = tf.keras.layers.Reshape((1731,), input_shape=(1731,1))(inputs1)
    
    tensor_1 = tf.keras.layers.Dense(20, activation='tanh')(inputs)
    tensor_1 = tf.keras.layers.Dense(1731, activation='tanh')(tensor_1)
    #tensor_2 = tf.keras.layers.BatchNormalization()(tensor_2)
    
    #tensor_2 = tf.keras.layers.Dropout(0.5)(tensor_2)
    
    product = tf.keras.layers.Multiply()([tensor_1, inputs1])
    
    outputs = tf.keras.layers.Dense(3, activation = 'linear')(product)
        
        # Finalize Model
    model = tf.keras.models.Model(inputs = inputs, outputs = outputs)

        # Compile model
    model.compile(loss = MeanAngErr,
                      optimizer= tf.keras.optimizers.Adam())
        
        # Show Model Summary
    model.summary()

    return model

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
model=create_model()

In [ ]:
#yval=loadY(525)
#xval=loadX(525)

#y=np.concatenate((loadY(520),loadY(521),loadY(522),loadY(523),loadY(524)),axis=0)
#x=np.concatenate((loadX(520),loadX(521),loadX(522),loadX(523),loadX(524)),axis=0)
y=np.concatenate((loadY(520),loadY(521),loadY(522),loadY(523),loadY(524),loadY(525)),axis=0)
x=np.concatenate((loadX(520),loadX(521),loadX(522),loadX(523),loadX(524),loadX(525)),axis=0)

In [ ]:
model.fit(x,y,epochs=10,batch_size=200)
model.fit(x,y,epochs=5,batch_size=2000)
model.fit(x,y,epochs=5,batch_size=20000)
model.fit(x,y,epochs=5,batch_size=200000)

In [ ]:
model.save_weights("model3.h5")